<a href="https://colab.research.google.com/github/vuongvmu/GCL_DemoCode/blob/main/rentina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Creat memory

In [ ]:
import os
import warnings

from retinaface import RetinaFace
import cv2
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context


from FDetector import FDetector
import numpy as np
import tensorflow as tf

# configurations
warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
# Limit the amount of reserved VRAM so that other scripts can be run in the same GPU as well
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

img1 = cv2.imread('d://222.png')
img2 = cv2.imread("d://111.png")
img1 = cv2.resize(img1, (640,480))
img2 = cv2.resize(img2, (640,480))

model = FDetector()
img = np.stack([img1, img2,img1, img1,img1, img1, img1, img1,img1, img1])

resp = model.detect_faces_on_frames(frames=img)


print(resp)

#FDetector

In [ ]:
import os
import threading
import time
from pathlib import Path
import gdown
import tensorflow as tf
from retinaface.commons.logger import Logger

from retinaface.model import retinaface_model
from retinaface.commons import preprocess, postprocess
from retinaface.commons.logger import Logger


# configurations
import numpy as np

tf_version = int(tf.__version__.split(".", maxsplit=1)[0])
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
        Input,
        BatchNormalization,
        ZeroPadding2D,
        Conv2D,
        ReLU,
        MaxPool2D,
        Add,
        UpSampling2D,
        concatenate,
        Softmax
)

class FDetector():
    def __init__(self):
        self.model = tf.function(
            self.__build_model(),
            input_signature=(tf.TensorSpec(shape=[None, None, None, 3], dtype=np.float32),)
        )
    def __build_model(self) -> Model:
        """
        Build RetinaFace model
        """
        data = Input(dtype=tf.float32, shape=(None, None, 3), name="data")

        bn_data = BatchNormalization(epsilon=1.9999999494757503e-05, name="bn_data", trainable=False)(
            data
        )

        conv0_pad = ZeroPadding2D(padding=tuple([3, 3]))(bn_data)

        conv0 = Conv2D(
            filters=64,
            kernel_size=(7, 7),
            name="conv0",
            strides=[2, 2],
            padding="VALID",
            use_bias=False,
        )(conv0_pad)

        bn0 = BatchNormalization(epsilon=1.9999999494757503e-05, name="bn0", trainable=False)(conv0)

        relu0 = ReLU(name="relu0")(bn0)

        pooling0_pad = ZeroPadding2D(padding=tuple([1, 1]))(relu0)

        pooling0 = MaxPool2D((3, 3), (2, 2), padding="valid", name="pooling0")(pooling0_pad)

        stage1_unit1_bn1 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage1_unit1_bn1", trainable=False
        )(pooling0)

        stage1_unit1_relu1 = ReLU(name="stage1_unit1_relu1")(stage1_unit1_bn1)

        stage1_unit1_conv1 = Conv2D(
            filters=64,
            kernel_size=(1, 1),
            name="stage1_unit1_conv1",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage1_unit1_relu1)

        stage1_unit1_sc = Conv2D(
            filters=256,
            kernel_size=(1, 1),
            name="stage1_unit1_sc",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage1_unit1_relu1)

        stage1_unit1_bn2 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage1_unit1_bn2", trainable=False
        )(stage1_unit1_conv1)

        stage1_unit1_relu2 = ReLU(name="stage1_unit1_relu2")(stage1_unit1_bn2)

        stage1_unit1_conv2_pad = ZeroPadding2D(padding=tuple([1, 1]))(stage1_unit1_relu2)

        stage1_unit1_conv2 = Conv2D(
            filters=64,
            kernel_size=(3, 3),
            name="stage1_unit1_conv2",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage1_unit1_conv2_pad)

        stage1_unit1_bn3 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage1_unit1_bn3", trainable=False
        )(stage1_unit1_conv2)

        stage1_unit1_relu3 = ReLU(name="stage1_unit1_relu3")(stage1_unit1_bn3)

        stage1_unit1_conv3 = Conv2D(
            filters=256,
            kernel_size=(1, 1),
            name="stage1_unit1_conv3",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage1_unit1_relu3)

        plus0_v1 = Add()([stage1_unit1_conv3, stage1_unit1_sc])

        stage1_unit2_bn1 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage1_unit2_bn1", trainable=False
        )(plus0_v1)

        stage1_unit2_relu1 = ReLU(name="stage1_unit2_relu1")(stage1_unit2_bn1)

        stage1_unit2_conv1 = Conv2D(
            filters=64,
            kernel_size=(1, 1),
            name="stage1_unit2_conv1",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage1_unit2_relu1)

        stage1_unit2_bn2 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage1_unit2_bn2", trainable=False
        )(stage1_unit2_conv1)

        stage1_unit2_relu2 = ReLU(name="stage1_unit2_relu2")(stage1_unit2_bn2)

        stage1_unit2_conv2_pad = ZeroPadding2D(padding=tuple([1, 1]))(stage1_unit2_relu2)

        stage1_unit2_conv2 = Conv2D(
            filters=64,
            kernel_size=(3, 3),
            name="stage1_unit2_conv2",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage1_unit2_conv2_pad)

        stage1_unit2_bn3 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage1_unit2_bn3", trainable=False
        )(stage1_unit2_conv2)

        stage1_unit2_relu3 = ReLU(name="stage1_unit2_relu3")(stage1_unit2_bn3)

        stage1_unit2_conv3 = Conv2D(
            filters=256,
            kernel_size=(1, 1),
            name="stage1_unit2_conv3",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage1_unit2_relu3)

        plus1_v2 = Add()([stage1_unit2_conv3, plus0_v1])

        stage1_unit3_bn1 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage1_unit3_bn1", trainable=False
        )(plus1_v2)

        stage1_unit3_relu1 = ReLU(name="stage1_unit3_relu1")(stage1_unit3_bn1)

        stage1_unit3_conv1 = Conv2D(
            filters=64,
            kernel_size=(1, 1),
            name="stage1_unit3_conv1",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage1_unit3_relu1)

        stage1_unit3_bn2 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage1_unit3_bn2", trainable=False
        )(stage1_unit3_conv1)

        stage1_unit3_relu2 = ReLU(name="stage1_unit3_relu2")(stage1_unit3_bn2)

        stage1_unit3_conv2_pad = ZeroPadding2D(padding=tuple([1, 1]))(stage1_unit3_relu2)

        stage1_unit3_conv2 = Conv2D(
            filters=64,
            kernel_size=(3, 3),
            name="stage1_unit3_conv2",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage1_unit3_conv2_pad)

        stage1_unit3_bn3 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage1_unit3_bn3", trainable=False
        )(stage1_unit3_conv2)

        stage1_unit3_relu3 = ReLU(name="stage1_unit3_relu3")(stage1_unit3_bn3)

        stage1_unit3_conv3 = Conv2D(
            filters=256,
            kernel_size=(1, 1),
            name="stage1_unit3_conv3",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage1_unit3_relu3)

        plus2 = Add()([stage1_unit3_conv3, plus1_v2])

        stage2_unit1_bn1 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage2_unit1_bn1", trainable=False
        )(plus2)

        stage2_unit1_relu1 = ReLU(name="stage2_unit1_relu1")(stage2_unit1_bn1)

        stage2_unit1_conv1 = Conv2D(
            filters=128,
            kernel_size=(1, 1),
            name="stage2_unit1_conv1",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage2_unit1_relu1)

        stage2_unit1_sc = Conv2D(
            filters=512,
            kernel_size=(1, 1),
            name="stage2_unit1_sc",
            strides=[2, 2],
            padding="VALID",
            use_bias=False,
        )(stage2_unit1_relu1)

        stage2_unit1_bn2 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage2_unit1_bn2", trainable=False
        )(stage2_unit1_conv1)

        stage2_unit1_relu2 = ReLU(name="stage2_unit1_relu2")(stage2_unit1_bn2)

        stage2_unit1_conv2_pad = ZeroPadding2D(padding=tuple([1, 1]))(stage2_unit1_relu2)

        stage2_unit1_conv2 = Conv2D(
            filters=128,
            kernel_size=(3, 3),
            name="stage2_unit1_conv2",
            strides=[2, 2],
            padding="VALID",
            use_bias=False,
        )(stage2_unit1_conv2_pad)

        stage2_unit1_bn3 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage2_unit1_bn3", trainable=False
        )(stage2_unit1_conv2)

        stage2_unit1_relu3 = ReLU(name="stage2_unit1_relu3")(stage2_unit1_bn3)

        stage2_unit1_conv3 = Conv2D(
            filters=512,
            kernel_size=(1, 1),
            name="stage2_unit1_conv3",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage2_unit1_relu3)

        plus3 = Add()([stage2_unit1_conv3, stage2_unit1_sc])

        stage2_unit2_bn1 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage2_unit2_bn1", trainable=False
        )(plus3)

        stage2_unit2_relu1 = ReLU(name="stage2_unit2_relu1")(stage2_unit2_bn1)

        stage2_unit2_conv1 = Conv2D(
            filters=128,
            kernel_size=(1, 1),
            name="stage2_unit2_conv1",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage2_unit2_relu1)

        stage2_unit2_bn2 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage2_unit2_bn2", trainable=False
        )(stage2_unit2_conv1)

        stage2_unit2_relu2 = ReLU(name="stage2_unit2_relu2")(stage2_unit2_bn2)

        stage2_unit2_conv2_pad = ZeroPadding2D(padding=tuple([1, 1]))(stage2_unit2_relu2)

        stage2_unit2_conv2 = Conv2D(
            filters=128,
            kernel_size=(3, 3),
            name="stage2_unit2_conv2",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage2_unit2_conv2_pad)

        stage2_unit2_bn3 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage2_unit2_bn3", trainable=False
        )(stage2_unit2_conv2)

        stage2_unit2_relu3 = ReLU(name="stage2_unit2_relu3")(stage2_unit2_bn3)

        stage2_unit2_conv3 = Conv2D(
            filters=512,
            kernel_size=(1, 1),
            name="stage2_unit2_conv3",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage2_unit2_relu3)

        plus4 = Add()([stage2_unit2_conv3, plus3])

        stage2_unit3_bn1 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage2_unit3_bn1", trainable=False
        )(plus4)

        stage2_unit3_relu1 = ReLU(name="stage2_unit3_relu1")(stage2_unit3_bn1)

        stage2_unit3_conv1 = Conv2D(
            filters=128,
            kernel_size=(1, 1),
            name="stage2_unit3_conv1",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage2_unit3_relu1)

        stage2_unit3_bn2 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage2_unit3_bn2", trainable=False
        )(stage2_unit3_conv1)

        stage2_unit3_relu2 = ReLU(name="stage2_unit3_relu2")(stage2_unit3_bn2)

        stage2_unit3_conv2_pad = ZeroPadding2D(padding=tuple([1, 1]))(stage2_unit3_relu2)

        stage2_unit3_conv2 = Conv2D(
            filters=128,
            kernel_size=(3, 3),
            name="stage2_unit3_conv2",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage2_unit3_conv2_pad)

        stage2_unit3_bn3 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage2_unit3_bn3", trainable=False
        )(stage2_unit3_conv2)

        stage2_unit3_relu3 = ReLU(name="stage2_unit3_relu3")(stage2_unit3_bn3)

        stage2_unit3_conv3 = Conv2D(
            filters=512,
            kernel_size=(1, 1),
            name="stage2_unit3_conv3",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage2_unit3_relu3)

        plus5 = Add()([stage2_unit3_conv3, plus4])

        stage2_unit4_bn1 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage2_unit4_bn1", trainable=False
        )(plus5)

        stage2_unit4_relu1 = ReLU(name="stage2_unit4_relu1")(stage2_unit4_bn1)

        stage2_unit4_conv1 = Conv2D(
            filters=128,
            kernel_size=(1, 1),
            name="stage2_unit4_conv1",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage2_unit4_relu1)

        stage2_unit4_bn2 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage2_unit4_bn2", trainable=False
        )(stage2_unit4_conv1)

        stage2_unit4_relu2 = ReLU(name="stage2_unit4_relu2")(stage2_unit4_bn2)

        stage2_unit4_conv2_pad = ZeroPadding2D(padding=tuple([1, 1]))(stage2_unit4_relu2)

        stage2_unit4_conv2 = Conv2D(
            filters=128,
            kernel_size=(3, 3),
            name="stage2_unit4_conv2",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage2_unit4_conv2_pad)

        stage2_unit4_bn3 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage2_unit4_bn3", trainable=False
        )(stage2_unit4_conv2)

        stage2_unit4_relu3 = ReLU(name="stage2_unit4_relu3")(stage2_unit4_bn3)

        stage2_unit4_conv3 = Conv2D(
            filters=512,
            kernel_size=(1, 1),
            name="stage2_unit4_conv3",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage2_unit4_relu3)

        plus6 = Add()([stage2_unit4_conv3, plus5])

        stage3_unit1_bn1 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage3_unit1_bn1", trainable=False
        )(plus6)

        stage3_unit1_relu1 = ReLU(name="stage3_unit1_relu1")(stage3_unit1_bn1)

        stage3_unit1_conv1 = Conv2D(
            filters=256,
            kernel_size=(1, 1),
            name="stage3_unit1_conv1",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage3_unit1_relu1)

        stage3_unit1_sc = Conv2D(
            filters=1024,
            kernel_size=(1, 1),
            name="stage3_unit1_sc",
            strides=[2, 2],
            padding="VALID",
            use_bias=False,
        )(stage3_unit1_relu1)

        stage3_unit1_bn2 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage3_unit1_bn2", trainable=False
        )(stage3_unit1_conv1)

        stage3_unit1_relu2 = ReLU(name="stage3_unit1_relu2")(stage3_unit1_bn2)

        stage3_unit1_conv2_pad = ZeroPadding2D(padding=tuple([1, 1]))(stage3_unit1_relu2)

        stage3_unit1_conv2 = Conv2D(
            filters=256,
            kernel_size=(3, 3),
            name="stage3_unit1_conv2",
            strides=[2, 2],
            padding="VALID",
            use_bias=False,
        )(stage3_unit1_conv2_pad)

        ssh_m1_red_conv = Conv2D(
            filters=256,
            kernel_size=(1, 1),
            name="ssh_m1_red_conv",
            strides=[1, 1],
            padding="VALID",
            use_bias=True,
        )(stage3_unit1_relu2)

        stage3_unit1_bn3 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage3_unit1_bn3", trainable=False
        )(stage3_unit1_conv2)

        ssh_m1_red_conv_bn = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="ssh_m1_red_conv_bn", trainable=False
        )(ssh_m1_red_conv)

        stage3_unit1_relu3 = ReLU(name="stage3_unit1_relu3")(stage3_unit1_bn3)

        ssh_m1_red_conv_relu = ReLU(name="ssh_m1_red_conv_relu")(ssh_m1_red_conv_bn)

        stage3_unit1_conv3 = Conv2D(
            filters=1024,
            kernel_size=(1, 1),
            name="stage3_unit1_conv3",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage3_unit1_relu3)

        plus7 = Add()([stage3_unit1_conv3, stage3_unit1_sc])

        stage3_unit2_bn1 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage3_unit2_bn1", trainable=False
        )(plus7)

        stage3_unit2_relu1 = ReLU(name="stage3_unit2_relu1")(stage3_unit2_bn1)

        stage3_unit2_conv1 = Conv2D(
            filters=256,
            kernel_size=(1, 1),
            name="stage3_unit2_conv1",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage3_unit2_relu1)

        stage3_unit2_bn2 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage3_unit2_bn2", trainable=False
        )(stage3_unit2_conv1)

        stage3_unit2_relu2 = ReLU(name="stage3_unit2_relu2")(stage3_unit2_bn2)

        stage3_unit2_conv2_pad = ZeroPadding2D(padding=tuple([1, 1]))(stage3_unit2_relu2)

        stage3_unit2_conv2 = Conv2D(
            filters=256,
            kernel_size=(3, 3),
            name="stage3_unit2_conv2",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage3_unit2_conv2_pad)

        stage3_unit2_bn3 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage3_unit2_bn3", trainable=False
        )(stage3_unit2_conv2)

        stage3_unit2_relu3 = ReLU(name="stage3_unit2_relu3")(stage3_unit2_bn3)

        stage3_unit2_conv3 = Conv2D(
            filters=1024,
            kernel_size=(1, 1),
            name="stage3_unit2_conv3",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage3_unit2_relu3)

        plus8 = Add()([stage3_unit2_conv3, plus7])

        stage3_unit3_bn1 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage3_unit3_bn1", trainable=False
        )(plus8)

        stage3_unit3_relu1 = ReLU(name="stage3_unit3_relu1")(stage3_unit3_bn1)

        stage3_unit3_conv1 = Conv2D(
            filters=256,
            kernel_size=(1, 1),
            name="stage3_unit3_conv1",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage3_unit3_relu1)

        stage3_unit3_bn2 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage3_unit3_bn2", trainable=False
        )(stage3_unit3_conv1)

        stage3_unit3_relu2 = ReLU(name="stage3_unit3_relu2")(stage3_unit3_bn2)

        stage3_unit3_conv2_pad = ZeroPadding2D(padding=tuple([1, 1]))(stage3_unit3_relu2)

        stage3_unit3_conv2 = Conv2D(
            filters=256,
            kernel_size=(3, 3),
            name="stage3_unit3_conv2",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage3_unit3_conv2_pad)

        stage3_unit3_bn3 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage3_unit3_bn3", trainable=False
        )(stage3_unit3_conv2)

        stage3_unit3_relu3 = ReLU(name="stage3_unit3_relu3")(stage3_unit3_bn3)

        stage3_unit3_conv3 = Conv2D(
            filters=1024,
            kernel_size=(1, 1),
            name="stage3_unit3_conv3",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage3_unit3_relu3)

        plus9 = Add()([stage3_unit3_conv3, plus8])

        stage3_unit4_bn1 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage3_unit4_bn1", trainable=False
        )(plus9)

        stage3_unit4_relu1 = ReLU(name="stage3_unit4_relu1")(stage3_unit4_bn1)

        stage3_unit4_conv1 = Conv2D(
            filters=256,
            kernel_size=(1, 1),
            name="stage3_unit4_conv1",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage3_unit4_relu1)

        stage3_unit4_bn2 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage3_unit4_bn2", trainable=False
        )(stage3_unit4_conv1)

        stage3_unit4_relu2 = ReLU(name="stage3_unit4_relu2")(stage3_unit4_bn2)

        stage3_unit4_conv2_pad = ZeroPadding2D(padding=tuple([1, 1]))(stage3_unit4_relu2)

        stage3_unit4_conv2 = Conv2D(
            filters=256,
            kernel_size=(3, 3),
            name="stage3_unit4_conv2",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage3_unit4_conv2_pad)

        stage3_unit4_bn3 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage3_unit4_bn3", trainable=False
        )(stage3_unit4_conv2)

        stage3_unit4_relu3 = ReLU(name="stage3_unit4_relu3")(stage3_unit4_bn3)

        stage3_unit4_conv3 = Conv2D(
            filters=1024,
            kernel_size=(1, 1),
            name="stage3_unit4_conv3",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage3_unit4_relu3)

        plus10 = Add()([stage3_unit4_conv3, plus9])

        stage3_unit5_bn1 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage3_unit5_bn1", trainable=False
        )(plus10)

        stage3_unit5_relu1 = ReLU(name="stage3_unit5_relu1")(stage3_unit5_bn1)

        stage3_unit5_conv1 = Conv2D(
            filters=256,
            kernel_size=(1, 1),
            name="stage3_unit5_conv1",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage3_unit5_relu1)

        stage3_unit5_bn2 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage3_unit5_bn2", trainable=False
        )(stage3_unit5_conv1)

        stage3_unit5_relu2 = ReLU(name="stage3_unit5_relu2")(stage3_unit5_bn2)

        stage3_unit5_conv2_pad = ZeroPadding2D(padding=tuple([1, 1]))(stage3_unit5_relu2)

        stage3_unit5_conv2 = Conv2D(
            filters=256,
            kernel_size=(3, 3),
            name="stage3_unit5_conv2",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage3_unit5_conv2_pad)

        stage3_unit5_bn3 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage3_unit5_bn3", trainable=False
        )(stage3_unit5_conv2)

        stage3_unit5_relu3 = ReLU(name="stage3_unit5_relu3")(stage3_unit5_bn3)

        stage3_unit5_conv3 = Conv2D(
            filters=1024,
            kernel_size=(1, 1),
            name="stage3_unit5_conv3",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage3_unit5_relu3)

        plus11 = Add()([stage3_unit5_conv3, plus10])

        stage3_unit6_bn1 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage3_unit6_bn1", trainable=False
        )(plus11)

        stage3_unit6_relu1 = ReLU(name="stage3_unit6_relu1")(stage3_unit6_bn1)

        stage3_unit6_conv1 = Conv2D(
            filters=256,
            kernel_size=(1, 1),
            name="stage3_unit6_conv1",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage3_unit6_relu1)

        stage3_unit6_bn2 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage3_unit6_bn2", trainable=False
        )(stage3_unit6_conv1)

        stage3_unit6_relu2 = ReLU(name="stage3_unit6_relu2")(stage3_unit6_bn2)

        stage3_unit6_conv2_pad = ZeroPadding2D(padding=tuple([1, 1]))(stage3_unit6_relu2)

        stage3_unit6_conv2 = Conv2D(
            filters=256,
            kernel_size=(3, 3),
            name="stage3_unit6_conv2",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage3_unit6_conv2_pad)

        stage3_unit6_bn3 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage3_unit6_bn3", trainable=False
        )(stage3_unit6_conv2)

        stage3_unit6_relu3 = ReLU(name="stage3_unit6_relu3")(stage3_unit6_bn3)

        stage3_unit6_conv3 = Conv2D(
            filters=1024,
            kernel_size=(1, 1),
            name="stage3_unit6_conv3",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage3_unit6_relu3)

        plus12 = Add()([stage3_unit6_conv3, plus11])

        stage4_unit1_bn1 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage4_unit1_bn1", trainable=False
        )(plus12)

        stage4_unit1_relu1 = ReLU(name="stage4_unit1_relu1")(stage4_unit1_bn1)

        stage4_unit1_conv1 = Conv2D(
            filters=512,
            kernel_size=(1, 1),
            name="stage4_unit1_conv1",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage4_unit1_relu1)

        stage4_unit1_sc = Conv2D(
            filters=2048,
            kernel_size=(1, 1),
            name="stage4_unit1_sc",
            strides=[2, 2],
            padding="VALID",
            use_bias=False,
        )(stage4_unit1_relu1)

        stage4_unit1_bn2 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage4_unit1_bn2", trainable=False
        )(stage4_unit1_conv1)

        stage4_unit1_relu2 = ReLU(name="stage4_unit1_relu2")(stage4_unit1_bn2)

        stage4_unit1_conv2_pad = ZeroPadding2D(padding=tuple([1, 1]))(stage4_unit1_relu2)

        stage4_unit1_conv2 = Conv2D(
            filters=512,
            kernel_size=(3, 3),
            name="stage4_unit1_conv2",
            strides=[2, 2],
            padding="VALID",
            use_bias=False,
        )(stage4_unit1_conv2_pad)

        ssh_c2_lateral = Conv2D(
            filters=256,
            kernel_size=(1, 1),
            name="ssh_c2_lateral",
            strides=[1, 1],
            padding="VALID",
            use_bias=True,
        )(stage4_unit1_relu2)

        stage4_unit1_bn3 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage4_unit1_bn3", trainable=False
        )(stage4_unit1_conv2)

        ssh_c2_lateral_bn = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="ssh_c2_lateral_bn", trainable=False
        )(ssh_c2_lateral)

        stage4_unit1_relu3 = ReLU(name="stage4_unit1_relu3")(stage4_unit1_bn3)

        ssh_c2_lateral_relu = ReLU(name="ssh_c2_lateral_relu")(ssh_c2_lateral_bn)

        stage4_unit1_conv3 = Conv2D(
            filters=2048,
            kernel_size=(1, 1),
            name="stage4_unit1_conv3",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage4_unit1_relu3)

        plus13 = Add()([stage4_unit1_conv3, stage4_unit1_sc])

        stage4_unit2_bn1 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage4_unit2_bn1", trainable=False
        )(plus13)

        stage4_unit2_relu1 = ReLU(name="stage4_unit2_relu1")(stage4_unit2_bn1)

        stage4_unit2_conv1 = Conv2D(
            filters=512,
            kernel_size=(1, 1),
            name="stage4_unit2_conv1",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage4_unit2_relu1)

        stage4_unit2_bn2 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage4_unit2_bn2", trainable=False
        )(stage4_unit2_conv1)

        stage4_unit2_relu2 = ReLU(name="stage4_unit2_relu2")(stage4_unit2_bn2)

        stage4_unit2_conv2_pad = ZeroPadding2D(padding=tuple([1, 1]))(stage4_unit2_relu2)

        stage4_unit2_conv2 = Conv2D(
            filters=512,
            kernel_size=(3, 3),
            name="stage4_unit2_conv2",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage4_unit2_conv2_pad)

        stage4_unit2_bn3 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage4_unit2_bn3", trainable=False
        )(stage4_unit2_conv2)

        stage4_unit2_relu3 = ReLU(name="stage4_unit2_relu3")(stage4_unit2_bn3)

        stage4_unit2_conv3 = Conv2D(
            filters=2048,
            kernel_size=(1, 1),
            name="stage4_unit2_conv3",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage4_unit2_relu3)

        plus14 = Add()([stage4_unit2_conv3, plus13])

        stage4_unit3_bn1 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage4_unit3_bn1", trainable=False
        )(plus14)

        stage4_unit3_relu1 = ReLU(name="stage4_unit3_relu1")(stage4_unit3_bn1)

        stage4_unit3_conv1 = Conv2D(
            filters=512,
            kernel_size=(1, 1),
            name="stage4_unit3_conv1",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage4_unit3_relu1)

        stage4_unit3_bn2 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage4_unit3_bn2", trainable=False
        )(stage4_unit3_conv1)

        stage4_unit3_relu2 = ReLU(name="stage4_unit3_relu2")(stage4_unit3_bn2)

        stage4_unit3_conv2_pad = ZeroPadding2D(padding=tuple([1, 1]))(stage4_unit3_relu2)

        stage4_unit3_conv2 = Conv2D(
            filters=512,
            kernel_size=(3, 3),
            name="stage4_unit3_conv2",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage4_unit3_conv2_pad)

        stage4_unit3_bn3 = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="stage4_unit3_bn3", trainable=False
        )(stage4_unit3_conv2)

        stage4_unit3_relu3 = ReLU(name="stage4_unit3_relu3")(stage4_unit3_bn3)

        stage4_unit3_conv3 = Conv2D(
            filters=2048,
            kernel_size=(1, 1),
            name="stage4_unit3_conv3",
            strides=[1, 1],
            padding="VALID",
            use_bias=False,
        )(stage4_unit3_relu3)

        plus15 = Add()([stage4_unit3_conv3, plus14])

        bn1 = BatchNormalization(epsilon=1.9999999494757503e-05, name="bn1", trainable=False)(plus15)

        relu1 = ReLU(name="relu1")(bn1)

        ssh_c3_lateral = Conv2D(
            filters=256,
            kernel_size=(1, 1),
            name="ssh_c3_lateral",
            strides=[1, 1],
            padding="VALID",
            use_bias=True,
        )(relu1)

        ssh_c3_lateral_bn = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="ssh_c3_lateral_bn", trainable=False
        )(ssh_c3_lateral)

        ssh_c3_lateral_relu = ReLU(name="ssh_c3_lateral_relu")(ssh_c3_lateral_bn)

        ssh_m3_det_conv1_pad = ZeroPadding2D(padding=tuple([1, 1]))(ssh_c3_lateral_relu)

        ssh_m3_det_conv1 = Conv2D(
            filters=256,
            kernel_size=(3, 3),
            name="ssh_m3_det_conv1",
            strides=[1, 1],
            padding="VALID",
            use_bias=True,
        )(ssh_m3_det_conv1_pad)

        ssh_m3_det_context_conv1_pad = ZeroPadding2D(padding=tuple([1, 1]))(ssh_c3_lateral_relu)

        ssh_m3_det_context_conv1 = Conv2D(
            filters=128,
            kernel_size=(3, 3),
            name="ssh_m3_det_context_conv1",
            strides=[1, 1],
            padding="VALID",
            use_bias=True,
        )(ssh_m3_det_context_conv1_pad)

        ssh_c3_up = UpSampling2D(size=(2, 2), interpolation="nearest", name="ssh_c3_up")(
            ssh_c3_lateral_relu
        )

        ssh_m3_det_conv1_bn = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="ssh_m3_det_conv1_bn", trainable=False
        )(ssh_m3_det_conv1)

        ssh_m3_det_context_conv1_bn = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="ssh_m3_det_context_conv1_bn", trainable=False
        )(ssh_m3_det_context_conv1)

        x1_shape = tf.shape(ssh_c3_up)
        x2_shape = tf.shape(ssh_c2_lateral_relu)
        offsets = [0, (x1_shape[1] - x2_shape[1]) // 2, (x1_shape[2] - x2_shape[2]) // 2, 0]
        size = [-1, x2_shape[1], x2_shape[2], -1]
        crop0 = tf.slice(ssh_c3_up, offsets, size, "crop0")

        ssh_m3_det_context_conv1_relu = ReLU(name="ssh_m3_det_context_conv1_relu")(
            ssh_m3_det_context_conv1_bn
        )

        plus0_v2 = Add()([ssh_c2_lateral_relu, crop0])

        ssh_m3_det_context_conv2_pad = ZeroPadding2D(padding=tuple([1, 1]))(
            ssh_m3_det_context_conv1_relu
        )

        ssh_m3_det_context_conv2 = Conv2D(
            filters=128,
            kernel_size=(3, 3),
            name="ssh_m3_det_context_conv2",
            strides=[1, 1],
            padding="VALID",
            use_bias=True,
        )(ssh_m3_det_context_conv2_pad)

        ssh_m3_det_context_conv3_1_pad = ZeroPadding2D(padding=tuple([1, 1]))(
            ssh_m3_det_context_conv1_relu
        )

        ssh_m3_det_context_conv3_1 = Conv2D(
            filters=128,
            kernel_size=(3, 3),
            name="ssh_m3_det_context_conv3_1",
            strides=[1, 1],
            padding="VALID",
            use_bias=True,
        )(ssh_m3_det_context_conv3_1_pad)

        ssh_c2_aggr_pad = ZeroPadding2D(padding=tuple([1, 1]))(plus0_v2)

        ssh_c2_aggr = Conv2D(
            filters=256,
            kernel_size=(3, 3),
            name="ssh_c2_aggr",
            strides=[1, 1],
            padding="VALID",
            use_bias=True,
        )(ssh_c2_aggr_pad)

        ssh_m3_det_context_conv2_bn = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="ssh_m3_det_context_conv2_bn", trainable=False
        )(ssh_m3_det_context_conv2)

        ssh_m3_det_context_conv3_1_bn = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="ssh_m3_det_context_conv3_1_bn", trainable=False
        )(ssh_m3_det_context_conv3_1)

        ssh_c2_aggr_bn = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="ssh_c2_aggr_bn", trainable=False
        )(ssh_c2_aggr)

        ssh_m3_det_context_conv3_1_relu = ReLU(name="ssh_m3_det_context_conv3_1_relu")(
            ssh_m3_det_context_conv3_1_bn
        )

        ssh_c2_aggr_relu = ReLU(name="ssh_c2_aggr_relu")(ssh_c2_aggr_bn)

        ssh_m3_det_context_conv3_2_pad = ZeroPadding2D(padding=tuple([1, 1]))(
            ssh_m3_det_context_conv3_1_relu
        )

        ssh_m3_det_context_conv3_2 = Conv2D(
            filters=128,
            kernel_size=(3, 3),
            name="ssh_m3_det_context_conv3_2",
            strides=[1, 1],
            padding="VALID",
            use_bias=True,
        )(ssh_m3_det_context_conv3_2_pad)

        ssh_m2_det_conv1_pad = ZeroPadding2D(padding=tuple([1, 1]))(ssh_c2_aggr_relu)

        ssh_m2_det_conv1 = Conv2D(
            filters=256,
            kernel_size=(3, 3),
            name="ssh_m2_det_conv1",
            strides=[1, 1],
            padding="VALID",
            use_bias=True,
        )(ssh_m2_det_conv1_pad)

        ssh_m2_det_context_conv1_pad = ZeroPadding2D(padding=tuple([1, 1]))(ssh_c2_aggr_relu)

        ssh_m2_det_context_conv1 = Conv2D(
            filters=128,
            kernel_size=(3, 3),
            name="ssh_m2_det_context_conv1",
            strides=[1, 1],
            padding="VALID",
            use_bias=True,
        )(ssh_m2_det_context_conv1_pad)

        ssh_m2_red_up = UpSampling2D(size=(2, 2), interpolation="nearest", name="ssh_m2_red_up")(
            ssh_c2_aggr_relu
        )

        ssh_m3_det_context_conv3_2_bn = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="ssh_m3_det_context_conv3_2_bn", trainable=False
        )(ssh_m3_det_context_conv3_2)

        ssh_m2_det_conv1_bn = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="ssh_m2_det_conv1_bn", trainable=False
        )(ssh_m2_det_conv1)

        ssh_m2_det_context_conv1_bn = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="ssh_m2_det_context_conv1_bn", trainable=False
        )(ssh_m2_det_context_conv1)

        x1_shape = tf.shape(ssh_m2_red_up)
        x2_shape = tf.shape(ssh_m1_red_conv_relu)
        offsets = [0, (x1_shape[1] - x2_shape[1]) // 2, (x1_shape[2] - x2_shape[2]) // 2, 0]
        size = [-1, x2_shape[1], x2_shape[2], -1]
        crop1 = tf.slice(ssh_m2_red_up, offsets, size, "crop1")

        ssh_m3_det_concat = concatenate(
            [ssh_m3_det_conv1_bn, ssh_m3_det_context_conv2_bn, ssh_m3_det_context_conv3_2_bn],
            3,
            name="ssh_m3_det_concat",
        )

        ssh_m2_det_context_conv1_relu = ReLU(name="ssh_m2_det_context_conv1_relu")(
            ssh_m2_det_context_conv1_bn
        )

        plus1_v1 = Add()([ssh_m1_red_conv_relu, crop1])

        ssh_m3_det_concat_relu = ReLU(name="ssh_m3_det_concat_relu")(ssh_m3_det_concat)

        ssh_m2_det_context_conv2_pad = ZeroPadding2D(padding=tuple([1, 1]))(
            ssh_m2_det_context_conv1_relu
        )

        ssh_m2_det_context_conv2 = Conv2D(
            filters=128,
            kernel_size=(3, 3),
            name="ssh_m2_det_context_conv2",
            strides=[1, 1],
            padding="VALID",
            use_bias=True,
        )(ssh_m2_det_context_conv2_pad)

        ssh_m2_det_context_conv3_1_pad = ZeroPadding2D(padding=tuple([1, 1]))(
            ssh_m2_det_context_conv1_relu
        )

        ssh_m2_det_context_conv3_1 = Conv2D(
            filters=128,
            kernel_size=(3, 3),
            name="ssh_m2_det_context_conv3_1",
            strides=[1, 1],
            padding="VALID",
            use_bias=True,
        )(ssh_m2_det_context_conv3_1_pad)

        ssh_c1_aggr_pad = ZeroPadding2D(padding=tuple([1, 1]))(plus1_v1)

        ssh_c1_aggr = Conv2D(
            filters=256,
            kernel_size=(3, 3),
            name="ssh_c1_aggr",
            strides=[1, 1],
            padding="VALID",
            use_bias=True,
        )(ssh_c1_aggr_pad)

        face_rpn_cls_score_stride32 = Conv2D(
            filters=4,
            kernel_size=(1, 1),
            name="face_rpn_cls_score_stride32",
            strides=[1, 1],
            padding="VALID",
            use_bias=True,
        )(ssh_m3_det_concat_relu)

        inter_1 = concatenate(
            [face_rpn_cls_score_stride32[:, :, :, 0], face_rpn_cls_score_stride32[:, :, :, 1]], axis=1
        )
        inter_2 = concatenate(
            [face_rpn_cls_score_stride32[:, :, :, 2], face_rpn_cls_score_stride32[:, :, :, 3]], axis=1
        )
        final = tf.stack([inter_1, inter_2])
        face_rpn_cls_score_reshape_stride32 = tf.transpose(
            final, (1, 2, 3, 0), name="face_rpn_cls_score_reshape_stride32"
        )

        face_rpn_bbox_pred_stride32 = Conv2D(
            filters=8,
            kernel_size=(1, 1),
            name="face_rpn_bbox_pred_stride32",
            strides=[1, 1],
            padding="VALID",
            use_bias=True,
        )(ssh_m3_det_concat_relu)

        face_rpn_landmark_pred_stride32 = Conv2D(
            filters=20,
            kernel_size=(1, 1),
            name="face_rpn_landmark_pred_stride32",
            strides=[1, 1],
            padding="VALID",
            use_bias=True,
        )(ssh_m3_det_concat_relu)

        ssh_m2_det_context_conv2_bn = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="ssh_m2_det_context_conv2_bn", trainable=False
        )(ssh_m2_det_context_conv2)

        ssh_m2_det_context_conv3_1_bn = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="ssh_m2_det_context_conv3_1_bn", trainable=False
        )(ssh_m2_det_context_conv3_1)

        ssh_c1_aggr_bn = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="ssh_c1_aggr_bn", trainable=False
        )(ssh_c1_aggr)

        ssh_m2_det_context_conv3_1_relu = ReLU(name="ssh_m2_det_context_conv3_1_relu")(
            ssh_m2_det_context_conv3_1_bn
        )

        ssh_c1_aggr_relu = ReLU(name="ssh_c1_aggr_relu")(ssh_c1_aggr_bn)

        face_rpn_cls_prob_stride32 = Softmax(name="face_rpn_cls_prob_stride32")(
            face_rpn_cls_score_reshape_stride32
        )

        input_shape = [tf.shape(face_rpn_cls_prob_stride32)[k] for k in range(4)]
        sz = tf.dtypes.cast(input_shape[1] / 2, dtype=tf.int32)
        inter_1 = face_rpn_cls_prob_stride32[:, 0:sz, :, 0]
        inter_2 = face_rpn_cls_prob_stride32[:, 0:sz, :, 1]
        inter_3 = face_rpn_cls_prob_stride32[:, sz:, :, 0]
        inter_4 = face_rpn_cls_prob_stride32[:, sz:, :, 1]
        final = tf.stack([inter_1, inter_3, inter_2, inter_4])
        face_rpn_cls_prob_reshape_stride32 = tf.transpose(
            final, (1, 2, 3, 0), name="face_rpn_cls_prob_reshape_stride32"
        )

        ssh_m2_det_context_conv3_2_pad = ZeroPadding2D(padding=tuple([1, 1]))(
            ssh_m2_det_context_conv3_1_relu
        )

        ssh_m2_det_context_conv3_2 = Conv2D(
            filters=128,
            kernel_size=(3, 3),
            name="ssh_m2_det_context_conv3_2",
            strides=[1, 1],
            padding="VALID",
            use_bias=True,
        )(ssh_m2_det_context_conv3_2_pad)

        ssh_m1_det_conv1_pad = ZeroPadding2D(padding=tuple([1, 1]))(ssh_c1_aggr_relu)

        ssh_m1_det_conv1 = Conv2D(
            filters=256,
            kernel_size=(3, 3),
            name="ssh_m1_det_conv1",
            strides=[1, 1],
            padding="VALID",
            use_bias=True,
        )(ssh_m1_det_conv1_pad)

        ssh_m1_det_context_conv1_pad = ZeroPadding2D(padding=tuple([1, 1]))(ssh_c1_aggr_relu)

        ssh_m1_det_context_conv1 = Conv2D(
            filters=128,
            kernel_size=(3, 3),
            name="ssh_m1_det_context_conv1",
            strides=[1, 1],
            padding="VALID",
            use_bias=True,
        )(ssh_m1_det_context_conv1_pad)

        ssh_m2_det_context_conv3_2_bn = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="ssh_m2_det_context_conv3_2_bn", trainable=False
        )(ssh_m2_det_context_conv3_2)

        ssh_m1_det_conv1_bn = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="ssh_m1_det_conv1_bn", trainable=False
        )(ssh_m1_det_conv1)

        ssh_m1_det_context_conv1_bn = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="ssh_m1_det_context_conv1_bn", trainable=False
        )(ssh_m1_det_context_conv1)

        ssh_m2_det_concat = concatenate(
            [ssh_m2_det_conv1_bn, ssh_m2_det_context_conv2_bn, ssh_m2_det_context_conv3_2_bn],
            3,
            name="ssh_m2_det_concat",
        )

        ssh_m1_det_context_conv1_relu = ReLU(name="ssh_m1_det_context_conv1_relu")(
            ssh_m1_det_context_conv1_bn
        )

        ssh_m2_det_concat_relu = ReLU(name="ssh_m2_det_concat_relu")(ssh_m2_det_concat)

        ssh_m1_det_context_conv2_pad = ZeroPadding2D(padding=tuple([1, 1]))(
            ssh_m1_det_context_conv1_relu
        )

        ssh_m1_det_context_conv2 = Conv2D(
            filters=128,
            kernel_size=(3, 3),
            name="ssh_m1_det_context_conv2",
            strides=[1, 1],
            padding="VALID",
            use_bias=True,
        )(ssh_m1_det_context_conv2_pad)

        ssh_m1_det_context_conv3_1_pad = ZeroPadding2D(padding=tuple([1, 1]))(
            ssh_m1_det_context_conv1_relu
        )

        ssh_m1_det_context_conv3_1 = Conv2D(
            filters=128,
            kernel_size=(3, 3),
            name="ssh_m1_det_context_conv3_1",
            strides=[1, 1],
            padding="VALID",
            use_bias=True,
        )(ssh_m1_det_context_conv3_1_pad)

        face_rpn_cls_score_stride16 = Conv2D(
            filters=4,
            kernel_size=(1, 1),
            name="face_rpn_cls_score_stride16",
            strides=[1, 1],
            padding="VALID",
            use_bias=True,
        )(ssh_m2_det_concat_relu)

        inter_1 = concatenate(
            [face_rpn_cls_score_stride16[:, :, :, 0], face_rpn_cls_score_stride16[:, :, :, 1]], axis=1
        )
        inter_2 = concatenate(
            [face_rpn_cls_score_stride16[:, :, :, 2], face_rpn_cls_score_stride16[:, :, :, 3]], axis=1
        )
        final = tf.stack([inter_1, inter_2])
        face_rpn_cls_score_reshape_stride16 = tf.transpose(
            final, (1, 2, 3, 0), name="face_rpn_cls_score_reshape_stride16"
        )

        face_rpn_bbox_pred_stride16 = Conv2D(
            filters=8,
            kernel_size=(1, 1),
            name="face_rpn_bbox_pred_stride16",
            strides=[1, 1],
            padding="VALID",
            use_bias=True,
        )(ssh_m2_det_concat_relu)

        face_rpn_landmark_pred_stride16 = Conv2D(
            filters=20,
            kernel_size=(1, 1),
            name="face_rpn_landmark_pred_stride16",
            strides=[1, 1],
            padding="VALID",
            use_bias=True,
        )(ssh_m2_det_concat_relu)

        ssh_m1_det_context_conv2_bn = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="ssh_m1_det_context_conv2_bn", trainable=False
        )(ssh_m1_det_context_conv2)

        ssh_m1_det_context_conv3_1_bn = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="ssh_m1_det_context_conv3_1_bn", trainable=False
        )(ssh_m1_det_context_conv3_1)

        ssh_m1_det_context_conv3_1_relu = ReLU(name="ssh_m1_det_context_conv3_1_relu")(
            ssh_m1_det_context_conv3_1_bn
        )

        face_rpn_cls_prob_stride16 = Softmax(name="face_rpn_cls_prob_stride16")(
            face_rpn_cls_score_reshape_stride16
        )

        input_shape = [tf.shape(face_rpn_cls_prob_stride16)[k] for k in range(4)]
        sz = tf.dtypes.cast(input_shape[1] / 2, dtype=tf.int32)
        inter_1 = face_rpn_cls_prob_stride16[:, 0:sz, :, 0]
        inter_2 = face_rpn_cls_prob_stride16[:, 0:sz, :, 1]
        inter_3 = face_rpn_cls_prob_stride16[:, sz:, :, 0]
        inter_4 = face_rpn_cls_prob_stride16[:, sz:, :, 1]
        final = tf.stack([inter_1, inter_3, inter_2, inter_4])
        face_rpn_cls_prob_reshape_stride16 = tf.transpose(
            final, (1, 2, 3, 0), name="face_rpn_cls_prob_reshape_stride16"
        )

        ssh_m1_det_context_conv3_2_pad = ZeroPadding2D(padding=tuple([1, 1]))(
            ssh_m1_det_context_conv3_1_relu
        )

        ssh_m1_det_context_conv3_2 = Conv2D(
            filters=128,
            kernel_size=(3, 3),
            name="ssh_m1_det_context_conv3_2",
            strides=[1, 1],
            padding="VALID",
            use_bias=True,
        )(ssh_m1_det_context_conv3_2_pad)

        ssh_m1_det_context_conv3_2_bn = BatchNormalization(
            epsilon=1.9999999494757503e-05, name="ssh_m1_det_context_conv3_2_bn", trainable=False
        )(ssh_m1_det_context_conv3_2)

        ssh_m1_det_concat = concatenate(
            [ssh_m1_det_conv1_bn, ssh_m1_det_context_conv2_bn, ssh_m1_det_context_conv3_2_bn],
            3,
            name="ssh_m1_det_concat",
        )

        ssh_m1_det_concat_relu = ReLU(name="ssh_m1_det_concat_relu")(ssh_m1_det_concat)
        face_rpn_cls_score_stride8 = Conv2D(
            filters=4,
            kernel_size=(1, 1),
            name="face_rpn_cls_score_stride8",
            strides=[1, 1],
            padding="VALID",
            use_bias=True,
        )(ssh_m1_det_concat_relu)

        inter_1 = concatenate(
            [face_rpn_cls_score_stride8[:, :, :, 0], face_rpn_cls_score_stride8[:, :, :, 1]], axis=1
        )
        inter_2 = concatenate(
            [face_rpn_cls_score_stride8[:, :, :, 2], face_rpn_cls_score_stride8[:, :, :, 3]], axis=1
        )
        final = tf.stack([inter_1, inter_2])
        face_rpn_cls_score_reshape_stride8 = tf.transpose(
            final, (1, 2, 3, 0), name="face_rpn_cls_score_reshape_stride8"
        )

        face_rpn_bbox_pred_stride8 = Conv2D(
            filters=8,
            kernel_size=(1, 1),
            name="face_rpn_bbox_pred_stride8",
            strides=[1, 1],
            padding="VALID",
            use_bias=True,
        )(ssh_m1_det_concat_relu)

        face_rpn_landmark_pred_stride8 = Conv2D(
            filters=20,
            kernel_size=(1, 1),
            name="face_rpn_landmark_pred_stride8",
            strides=[1, 1],
            padding="VALID",
            use_bias=True,
        )(ssh_m1_det_concat_relu)

        face_rpn_cls_prob_stride8 = Softmax(name="face_rpn_cls_prob_stride8")(
            face_rpn_cls_score_reshape_stride8
        )

        input_shape = [tf.shape(face_rpn_cls_prob_stride8)[k] for k in range(4)]
        sz = tf.dtypes.cast(input_shape[1] / 2, dtype=tf.int32)
        inter_1 = face_rpn_cls_prob_stride8[:, 0:sz, :, 0]
        inter_2 = face_rpn_cls_prob_stride8[:, 0:sz, :, 1]
        inter_3 = face_rpn_cls_prob_stride8[:, sz:, :, 0]
        inter_4 = face_rpn_cls_prob_stride8[:, sz:, :, 1]
        final = tf.stack([inter_1, inter_3, inter_2, inter_4])
        face_rpn_cls_prob_reshape_stride8 = tf.transpose(
            final, (1, 2, 3, 0), name="face_rpn_cls_prob_reshape_stride8"
        )

        model = Model(
            inputs=data,
            outputs=[
                face_rpn_cls_prob_reshape_stride32,
                face_rpn_bbox_pred_stride32,
                face_rpn_landmark_pred_stride32,
                face_rpn_cls_prob_reshape_stride16,
                face_rpn_bbox_pred_stride16,
                face_rpn_landmark_pred_stride16,
                face_rpn_cls_prob_reshape_stride8,
                face_rpn_bbox_pred_stride8,
                face_rpn_landmark_pred_stride8,
            ],
        )
        model.load_weights("d://models/retinaface.h5")

        return model

    def __determize_faces(self,img, net_out):
        start = time.time()
        threshold = 0.9
        allow_upscaling = True
        resp = {}
       # ---------------------------
        nms_threshold = 0.4
        decay4 = 0.5
        _feat_stride_fpn = [32, 16, 8]
        _anchors_fpn = {
            "stride32": np.array(
                [[-248.0, -248.0, 263.0, 263.0],[-120.0, -120.0, 135.0, 135.0]], dtype=np.float32 ),
            "stride16": np.array(
                [[-56.0, -56.0, 71.0, 71.0], [-24.0, -24.0, 39.0, 39.0]], dtype=np.float32 ),
            "stride8": np.array(
                [[-8.0, -8.0, 23.0, 23.0], [0.0, 0.0, 15.0, 15.0]], dtype=np.float32),
        }

        _num_anchors = {"stride32": 2, "stride16": 2, "stride8": 2}
        # ---------------------------
        proposals_list = []
        scores_list = []
        landmarks_list = []

        im_tensor, im_info, im_scale = preprocess.preprocess_image(img, allow_upscaling)


        net_out = [elt.numpy() for elt in net_out]
        sym_idx = 0

        for _, s in enumerate(_feat_stride_fpn):
            # _key = f"stride{s}"
            scores = net_out[sym_idx]
            scores = scores[:, :, :, _num_anchors[f"stride{s}"]:]

            bbox_deltas = net_out[sym_idx + 1]
            height, width = bbox_deltas.shape[1], bbox_deltas.shape[2]

            A = _num_anchors[f"stride{s}"]
            K = height * width
            anchors_fpn = _anchors_fpn[f"stride{s}"]
            anchors = postprocess.anchors_plane(height, width, s, anchors_fpn)
            anchors = anchors.reshape((K * A, 4))
            scores = scores.reshape((-1, 1))

            bbox_stds = [1.0, 1.0, 1.0, 1.0]
            bbox_pred_len = bbox_deltas.shape[3] // A
            bbox_deltas = bbox_deltas.reshape((-1, bbox_pred_len))
            bbox_deltas[:, 0::4] = bbox_deltas[:, 0::4] * bbox_stds[0]
            bbox_deltas[:, 1::4] = bbox_deltas[:, 1::4] * bbox_stds[1]
            bbox_deltas[:, 2::4] = bbox_deltas[:, 2::4] * bbox_stds[2]
            bbox_deltas[:, 3::4] = bbox_deltas[:, 3::4] * bbox_stds[3]
            proposals = postprocess.bbox_pred(anchors, bbox_deltas)

            proposals = postprocess.clip_boxes(proposals, im_info[:2])

            if s == 4 and decay4 < 1.0:
                scores *= decay4

            scores_ravel = scores.ravel()
            order = np.where(scores_ravel >= threshold)[0]
            proposals = proposals[order, :]
            scores = scores[order]

            proposals[:, 0:4] /= im_scale
            proposals_list.append(proposals)
            scores_list.append(scores)

            landmark_deltas = net_out[sym_idx + 2]
            landmark_pred_len = landmark_deltas.shape[3] // A
            landmark_deltas = landmark_deltas.reshape((-1, 5, landmark_pred_len // 5))
            landmarks = postprocess.landmark_pred(anchors, landmark_deltas)
            landmarks = landmarks[order, :]

            landmarks[:, :, 0:2] /= im_scale
            landmarks_list.append(landmarks)
            sym_idx += 3

        proposals = np.vstack(proposals_list)
        if proposals.shape[0] == 0:
            return resp

        scores = np.vstack(scores_list)
        scores_ravel = scores.ravel()
        order = scores_ravel.argsort()[::-1]
        proposals = proposals[order, :]
        scores = scores[order]
        landmarks = np.vstack(landmarks_list)
        landmarks = landmarks[order].astype(np.float32, copy=False)
        pre_det = np.hstack((proposals[:, 0:4], scores)).astype(np.float32, copy=False)
        keep = postprocess.cpu_nms(pre_det, nms_threshold)
        det = np.hstack((pre_det, proposals[:, 4:]))
        det = det[keep, :]
        landmarks = landmarks[keep]
        for idx, face in enumerate(det):
            label = "face_" + str(idx + 1)
            resp[label] = {}
            resp[label]["score"] = face[4]
            resp[label]["facial_area"] = list(face[0:4].astype(int))
            resp[label]["landmarks"] = {}
            resp[label]["landmarks"]["right_eye"] = list(landmarks[idx][0])
            resp[label]["landmarks"]["left_eye"] = list(landmarks[idx][1])
            resp[label]["landmarks"]["nose"] = list(landmarks[idx][2])
            resp[label]["landmarks"]["mouth_right"] = list(landmarks[idx][3])
            resp[label]["landmarks"]["mouth_left"] = list(landmarks[idx][4])
        print(resp)
        end = time.time()
        print(f" total  : {(end - start) * 1000} ms")

    def detect_faces_on_frames(self, frames:np.array):

        for index in range (10):
            time.sleep(0.001)
            n_frames = frames.shape[0]
            frames_out = self.model(frames)
            for i in range(n_frames):
                net_i_out = [
                    tf.expand_dims(i_out[i], axis=0)
                    for i_out in frames_out
                ]
                thread = threading.Thread(
                    target=self.__determize_faces, args=(frames[i][:][:][:],net_i_out)
                ).start()
        print("finished")






